In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Create a folder in your Google Drive (if it doesn't exist)
!mkdir -p /content/drive/MyDrive/TravelBO/Levy4
print("Below is a list of finished benchmarks.")
!ls /content/drive/MyDrive/TravelBO/Levy4

In [ ]:
!pip install -q botorch==0.13.0

In [ ]:
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/TUCB.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/UCB.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/TTS.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/TS.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/GS.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/RS.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/TuRBO.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/LHS.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/SnAKe.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/MINI.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/LA.py
!wget https://raw.githubusercontent.com/Chen-Qiyuan/Travel-BO/refs/heads/main/main/levy4/figure.py

In [ ]:
import os
import subprocess
import shutil
import sys

def generate_if_needed(algo_name):
  script_path = algo_name + ".py"
  output_filename1 = algo_name + "_travel.xlsx"
  output_filename2 = algo_name + "_reg.xlsx"
  destination_folder = "/content/drive/MyDrive/TravelBO/Levy4/"

  # Check if the file already exists
  if os.path.exists(os.path.join(destination_folder, output_filename1)):
    print(f"'{algo_name}' already finished. Skipping.")
    return

  # If the file does not exist, proceed with generation and copying
  print(f"Running {algo_name}...")

  # Use subprocess to run the external script
  subprocess.run([sys.executable, script_path], check=True, text=True)

  print(f"Copying '{output_filename1}' to Google Drive...")
  os.makedirs(destination_folder, exist_ok=True)
  shutil.copy(os.path.join('/content/', output_filename1), os.path.join(destination_folder, output_filename1))

  print(f"Copying '{output_filename2}' to Google Drive...")
  os.makedirs(destination_folder, exist_ok=True)
  shutil.copy(os.path.join('/content/', output_filename2), os.path.join(destination_folder, output_filename2))

for algo_name in ["TS", "TTS", "UCB", "TUCB", "MINI", "RS", "LHS", "TuRBO", "GS", "SnAKe", "LA"]:
  generate_if_needed(algo_name)

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
# Below is copy and pasted from plot.ipynb
def plot_from_files():
    plt.rcParams['axes.prop_cycle'] = cycler('color', plt.colormaps['tab20'].colors)
    workspace = "/content/drive/MyDrive/TravelBO/Levy4"
    desired_order = [
        "GS", "LA", "LHS", "MINI", "RS", "SnAKe", "TS", "TTS", "TUCB", "TuRBO", "UCB"
    ]

    print("--- Generating Regret Plot ---")

    # Get all files from the directory
    all_files_reg = glob.glob(os.path.join(workspace, '*_reg.xlsx'))
    print(f"Found {len(all_files_reg)} regret files.")

    # Create a new list of files sorted according to desired_order
    reordered_files_reg = []
    for algo_name in desired_order:
        expected_file = os.path.join(workspace, f"{algo_name}_reg.xlsx")
        if expected_file in all_files_reg:
            reordered_files_reg.append(expected_file)

    # Plot using the reordered list
    fig1, ax1 = plt.subplots(figsize=(10, 6))

    for file in reordered_files_reg:
        df = pd.read_excel(file)[:200]
        cumulative_regret = df.cumsum()
        t = cumulative_regret.index + 1
        average_regret = cumulative_regret.div(t, axis=0)
        mean = average_regret.mean(axis=1)
        std = average_regret.std(axis=1) / np.sqrt(10)

        # Extract name and plot with the 'label' keyword
        algorithm_name = os.path.basename(file).replace('_reg.xlsx', '')
        ax1.plot(mean, label=algorithm_name)
        ax1.fill_between(range(len(mean)), mean - std, mean + std, alpha=0.2)

    # Format plot
    ax1.set_xlabel('Round (t)', fontsize=16)
    ax1.set_ylabel('Log Average Cumulative Regret log$(R_t/t)$', fontsize=16)
    ax1.legend(fontsize=14, loc='lower left')
    ax1.set_xlim(0, 200)
    ax1.grid(True)
    ax1.set_yscale("log")

    for tick in ax1.get_yticklabels(which='both'):
        tick.set_rotation(90)
        tick.set_ha('center')
        tick.set_va('center')

    plt.tight_layout()
    plt.savefig(os.path.join("Figures", os.path.basename(workspace.rstrip('/\\')) + "_regret.pdf"), dpi=600)
    plt.show()

    print("\n--- Generating Travel Cost Plot ---")

    # Get all travel files
    all_files_travel = glob.glob(os.path.join(workspace, '*_travel.xlsx'))
    print(f"Found {len(all_files_travel)} travel files.")

    # Create a reordered list for the travel files
    reordered_files_travel = []
    for algo_name in desired_order:
        expected_file = os.path.join(workspace, f"{algo_name}_travel.xlsx")
        if expected_file in all_files_travel:
            reordered_files_travel.append(expected_file)

    # Plot using the reordered list
    fig2, ax2 = plt.subplots(figsize=(10, 6))

    for file in reordered_files_travel:
        df = pd.read_excel(file)[:200]
        cumulative_regret = df.cumsum()
        t = cumulative_regret.index + 1
        average_regret = cumulative_regret.div(t, axis=0)
        mean = average_regret.mean(axis=1)
        std = average_regret.std(axis=1) / np.sqrt(10)

        # Extract name and plot with the 'label' keyword
        algorithm_name = os.path.basename(file).replace('_travel.xlsx', '')
        ax2.plot(mean, label=algorithm_name)
        ax2.fill_between(range(len(mean)), mean - std, mean + std, alpha=0.2)

    # Format plot
    ax2.set_xlabel('Round (t)', fontsize=16)
    ax2.set_ylabel('Average Movement Cost $C_t/t$', fontsize=16)
    ax2.legend(fontsize=14, loc='lower left')
    ax2.set_xlim(0, 200)
    ax2.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join("Figures", os.path.basename(workspace.rstrip('/\\')) + "_cost.pdf"), dpi=600)
    plt.show()

# Create the "Figures" directory if it doesn't exist
os.makedirs("Figures", exist_ok=True)

plot_from_files()
print("All finished.")